In [0]:
# https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149
import tensorflow as tf
import numpy as np

In [0]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.


# doesn't work with negative values smaller than around this a and b.
a = -2.5
b = -3.95
c = 17
d = 0.001
bandits = [[a, b, c, d],[b, c, d, a],[c, d, a, b],[d, a, b, c]]
num_bandits = len(bandits)
cycle = 0
def pullBandit(action):
    global cycle
    #Get a random number.
    result = np.random.randn(1)
    if result > bandits[cycle & 3][action]:
        #return a positive reward.
        k = 1
        if (cycle & 3) >= 1: k /= cycle
    else:
        #return a negative reward.
        k = -1
        if (cycle & 3) >= 1: k /= cycle
    
    cycle += 1
    return k

In [0]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [79]:
total_episodes = 50000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(action) #Get our reward from picking one of the bandits.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 1000 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits[0])):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [0. 1. 0. 0.]
Running reward for the 4 bandits: [  9.99233922 226.46702874  -5.99378022   1.98235498]
Running reward for the 4 bandits: [ 14.98992898 460.4744286  -10.98123336   5.99453335]
Running reward for the 4 bandits: [ 22.98947222 687.49068704 -16.9785675    4.99365627]
Running reward for the 4 bandits: [ 27.98783854 921.49593733 -19.97492744   8.99531784]
Running reward for the 4 bandits: [  32.9884064  1151.49731196  -25.9721688    13.99683899]
Running reward for the 4 bandits: [  35.98778698 1385.49195502  -31.9690362    14.99845981]
Running reward for the 4 bandits: [  39.98714588 1621.48957191  -34.96685607   16.00016556]
Running reward for the 4 bandits: [  46.98739435 1850.4858404   -40.9653958    16.00178585]
Running reward for the 4 bandits: [  52.98760953 2083.48415927  -46.96456754   17.00236886]
Running reward for the 4 bandits: [  60.98709932 2308.48470555  -58.96371601   14.00321324]
Running reward for the 4 bandits: [  66.98767685

Running reward for the 4 bandits: [  250.98760864 10664.48034032  -299.94495509    15.01230741]
Running reward for the 4 bandits: [  255.98760936 10901.48068987  -304.9446755     16.01241528]
Running reward for the 4 bandits: [  268.98758879 11125.48064538  -314.94442356    15.01239436]
Running reward for the 4 bandits: [  275.98763049 11355.48029527  -322.94423885    16.01264165]
The agent thinks bandit 2 is the most promising....
...and it was right!
